In [1]:
# 02_Feature_Engineering.ipynb — starter block
# 1) imports and load raw data
import os
import pandas as pd
import numpy as np
import pickle

project_root = os.path.abspath("..")   # notebooks/ -> project root
print("Project root:", project_root)

raw_path = os.path.join(project_root, "data", "raw", "loan.csv")   # adjust if needed
print("Loading:", raw_path)

# load (large file) — may take time
df = pd.read_csv(raw_path, low_memory=False)
print("Loaded shape:", df.shape)

# 2) create target column 'default' (bad=1, good=0)
bad_status = [
    "Charged Off",
    "Default",
    "Late (31-120 days)",
    "Late (16-30 days)",
    "Does not meet the credit policy. Status: Charged Off",
    "Does not meet the credit policy. Status:Charged Off"  # include both variants
]
# robust mapping (strip whitespace)
df['loan_status'] = df['loan_status'].astype(str).str.strip()
df['default'] = df['loan_status'].apply(lambda x: 1 if x in bad_status else 0)
print("Target distribution:\n", df['default'].value_counts())

# 3) drop leakage / id / hardship / settlement / sec_app columns (conservative list)
cols_to_drop = [
    # IDs & text
    'id','member_id','emp_title','title','url','desc','zip_code',
    # date-like or post-event leakage
    'issue_d','earliest_cr_line','last_pymnt_d','next_pymnt_d','last_credit_pull_d',
    # payment leakage
    'out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp',
    'total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_amnt',
    'loan_status',  # original label column
    # hardship & settlement (very sparse)
    'orig_projected_additional_accrued_interest','hardship_amount','hardship_status','payment_plan_start_date',
    'hardship_last_payment_amount','hardship_payoff_balance_amount','hardship_loan_status','hardship_type',
    'hardship_dpd','hardship_start_date','hardship_end_date','hardship_reason','hardship_length','deferral_term',
    'settlement_term','debt_settlement_flag_date','settlement_status','settlement_date','settlement_amount',
    'settlement_percentage',
    # secondary applicant fields (very sparse / often blank)
    'sec_app_mths_since_last_major_derog','sec_app_revol_util','revol_bal_joint','sec_app_earliest_cr_line',
    'sec_app_inq_last_6mths','sec_app_chargeoff_within_12_mths','sec_app_collections_12_mths_ex_med',
    'sec_app_open_act_il','sec_app_num_rev_accts','sec_app_open_acc','sec_app_mort_acc',
    'verification_status_joint','dti_joint','annual_inc_joint'
]

cols_present_to_drop = [c for c in cols_to_drop if c in df.columns]
print("Dropping columns (count):", len(cols_present_to_drop))
df = df.drop(columns=cols_present_to_drop)
print("After drop shape:", df.shape)

# 4) quick missing-value strategy (numeric -> median, categorical -> mode)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
print("# numeric cols:", len(num_cols), " # categorical cols:", len(cat_cols))

# Fill numeric with median
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Fill categorical with mode (most frequent)
for c in cat_cols:
    if df[c].isna().sum() > 0:
        df[c] = df[c].fillna(df[c].mode().iloc[0])

# confirm no missing values
print("Missing values left:", df.isna().sum().sum())

# 5) feature engineering: simple, high-impact features
# credit_utilization = revol_bal / total_rev_hi_lim (if exists)
if 'revol_bal' in df.columns and 'total_rev_hi_lim' in df.columns:
    df['credit_utilization'] = df['revol_bal'] / (df['total_rev_hi_lim'].replace(0, np.nan))
    df['credit_utilization'] = df['credit_utilization'].fillna(0)

# payment_to_income = installment / annual_inc
if 'installment' in df.columns and 'annual_inc' in df.columns:
    df['payment_to_income'] = df['installment'] / (df['annual_inc'].replace(0, np.nan))
    df['payment_to_income'] = df['payment_to_income'].fillna(0)

# funded_ratio = funded_amnt / loan_amnt (if both exist)
if 'funded_amnt' in df.columns and 'loan_amnt' in df.columns:
    df['funded_ratio'] = df['funded_amnt'] / df['loan_amnt']

# convert emp_length to numeric if present (e.g., '10+ years', '< 1 year', 'n/a')
def emp_to_num(x):
    try:
        if pd.isna(x): return np.nan
        s = str(x).strip()
        if s == 'n/a': return np.nan
        if '+' in s:
            return float(s.split('+')[0])
        if '<' in s:
            return 0.0
        return float(''.join(ch for ch in s if ch.isdigit()) or np.nan)
    except:
        return np.nan

if 'emp_length' in df.columns:
    df['emp_length_num'] = df['emp_length'].apply(emp_to_num)
    df['emp_length_num'] = df['emp_length_num'].fillna(df['emp_length_num'].median())

# 6) ordinal encode grade & sub_grade
if 'grade' in df.columns:
    grade_mapping = {'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7}
    df['grade_ord'] = df['grade'].map(grade_mapping)

if 'sub_grade' in df.columns:
    # convert 'A1' -> 1.1, 'B3' -> 2.3
    def subgrade_to_num(s):
        try:
            s = str(s).strip()
            return grade_mapping[s[0]] + (int(s[1]) / 10)
        except:
            return np.nan
    df['sub_grade_ord'] = df['sub_grade'].apply(subgrade_to_num)
    df['sub_grade_ord'] = df['sub_grade_ord'].fillna(df['sub_grade_ord'].median())

# 7) choose how to encode remaining categorical cols intelligently:
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
print("Remaining object cols before encoding:", cat_cols)

# Strategy:
# - If a categorical column has low cardinality (<50), one-hot it.
# - If high-cardinality, convert to category codes to save memory.
to_one_hot = [c for c in cat_cols if df[c].nunique() < 50]
to_cat_code = [c for c in cat_cols if df[c].nunique() >= 50]

print("One-hot columns (count):", len(to_one_hot))
print("Category-code columns (count):", len(to_cat_code))

# apply category codes for high-cardinality to save memory
for c in to_cat_code:
    df[c] = df[c].astype('category').cat.codes  # -1 means NaN (should be none)

# one-hot low-cardinality columns
if len(to_one_hot) > 0:
    df = pd.get_dummies(df, columns=to_one_hot, drop_first=True)
    print("After one-hot shape:", df.shape)

# 8) final housekeeping: ensure 'default' exists and save processed data
print("Final shape:", df.shape)
processed_dir = os.path.join(project_root, "data", "processed")
os.makedirs(processed_dir, exist_ok=True)

clean_file = os.path.join(processed_dir, "loan_cleaned_for_model.csv")
df.to_csv(clean_file, index=False)
print("Saved cleaned CSV to:", clean_file)

# also save a smaller pickle for faster reload (optional)
pkl_file = os.path.join(processed_dir, "loan_cleaned_for_model.pkl")
with open(pkl_file, "wb") as f:
    pickle.dump(df, f)
print("Saved cleaned pickle to:", pkl_file)

# 9) print head and columns sample
print("Columns sample:", df.columns[:40].tolist())
df.head()


Project root: C:\Credit-risk-scoring
Loading: C:\Credit-risk-scoring\data\raw\loan.csv
Loaded shape: (2260668, 145)
Target distribution:
 default
0    1972587
1     288081
Name: count, dtype: int64
Dropping columns (count): 57
After drop shape: (2260668, 89)
# numeric cols: 75  # categorical cols: 14
Missing values left: 0
Remaining object cols before encoding: ['term', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'verification_status', 'pymnt_plan', 'purpose', 'addr_state', 'initial_list_status', 'application_type', 'hardship_flag', 'disbursement_method', 'debt_settlement_flag']
One-hot columns (count): 13
Category-code columns (count): 1
After one-hot shape: (2260668, 159)
Final shape: (2260668, 159)
Saved cleaned CSV to: C:\Credit-risk-scoring\data\processed\loan_cleaned_for_model.csv
Saved cleaned pickle to: C:\Credit-risk-scoring\data\processed\loan_cleaned_for_model.pkl
Columns sample: ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'annual_inc

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,addr_state,dti,delinq_2yrs,inq_last_6mths,...,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_w,application_type_Joint App,hardship_flag_Y,disbursement_method_DirectPay,debt_settlement_flag_Y
0,2500,2500,2500.0,13.56,84.92,55000.0,34,18.24,0.0,1.0,...,False,False,False,False,False,True,False,False,False,False
1,30000,30000,30000.0,18.94,777.23,90000.0,18,26.52,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False
2,5000,5000,5000.0,17.97,180.69,59280.0,22,10.51,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False
3,4000,4000,4000.0,18.94,146.51,92000.0,47,16.74,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False
4,30000,30000,30000.0,16.14,731.78,57250.0,20,26.35,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False


In [5]:
!dir "../data/processed"


 Volume in drive C is Windows
 Volume Serial Number is 2EA3-CE1B

 Directory of C:\Credit-risk-scoring\data\processed

11-12-2025  23:46    <DIR>          .
11-12-2025  22:44    <DIR>          ..
11-12-2025  23:46     2,001,513,625 loan_cleaned_for_model.csv
11-12-2025  23:46     1,641,250,225 loan_cleaned_for_model.pkl
               2 File(s)  3,642,763,850 bytes
               2 Dir(s)  151,600,525,312 bytes free


In [6]:
df.head()



,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,addr_state,dti,delinq_2yrs,inq_last_6mths,...,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_w,application_type_Joint App,hardship_flag_Y,disbursement_method_DirectPay,debt_settlement_flag_Y
0,2500,2500,2500.0,13.56,84.92,55000.0,34,18.24,0.0,1.0,...,False,False,False,False,False,True,False,False,False,False
1,30000,30000,30000.0,18.94,777.23,90000.0,18,26.52,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False
2,5000,5000,5000.0,17.97,180.69,59280.0,22,10.51,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False
3,4000,4000,4000.0,18.94,146.51,92000.0,47,16.74,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False
4,30000,30000,30000.0,16.14,731.78,57250.0,20,26.35,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False


In [7]:
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260668 entries, 0 to 2260667
Columns: 159 entries, loan_amnt to debt_settlement_flag_Y
dtypes: bool(77), float64(75), int64(6), int8(1)
memory usage: 1.5 GB


In [8]:
df['default'].value_counts()


default
0    1972587
1     288081
Name: count, dtype: int64